In [1]:
import streamlit as st
import cv2
import numpy as np
import joblib
from skimage.feature import hog, local_binary_pattern
from sklearn.preprocessing import StandardScaler
from collections import Counter
from PIL import Image

# Load the trained models and label encoder
svm_model = joblib.load("best_svm_model.pkl")
dt_model = joblib.load("best_dt_model.pkl")
rf_model = joblib.load("best_rf_model.pkl")
label_encoder = joblib.load("label_encoder.pkl")

# Preprocessing functions
def preprocess_image(image):
    img = np.array(image)
    img_resized = cv2.resize(img, (128, 128))
    img_gray = cv2.cvtColor(img_resized, cv2.COLOR_RGB2GRAY)
    return img_resized, img_gray

def extract_color_histogram(img, bins=(16, 16, 16)):
    hist = cv2.calcHist([img], [0, 1, 2], None, bins, [0, 256, 0, 256, 0, 256])
    cv2.normalize(hist, hist)
    return hist.flatten()

def extract_hog_features(img_gray):
    features = hog(img_gray, pixels_per_cell=(8, 8), cells_per_block=(3, 3),
                   visualize=False, block_norm='L2-Hys')
    return features

def extract_lbp_features(img_gray, radius=3, n_points=24):
    lbp = local_binary_pattern(img_gray, n_points, radius, method='uniform')
    hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, n_points + 3), range=(0, n_points + 2))
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-6)
    return hist

# Main app
st.title("Tomato Leaf Disease Classifier 🍅")
st.write("Upload a tomato leaf image and get the predicted infection type using an ensemble of models.")

uploaded_file = st.file_uploader("Choose a tomato leaf image...", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    image = Image.open(uploaded_file).convert("RGB")
    st.image(image, caption='Uploaded Image', use_column_width=True)

    # Preprocess and extract features
    img_resized, img_gray = preprocess_image(image)
    hist = extract_color_histogram(img_resized)
    hog_feat = extract_hog_features(img_gray)
    lbp_feat = extract_lbp_features(img_gray)

    # Combine features and scale
    features = np.hstack((hog_feat, hist, lbp_feat)).reshape(1, -1)
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features)

    # Get predictions
    pred_svm = svm_model.predict(features_scaled)[0]
    pred_dt = dt_model.predict(features_scaled)[0]
    pred_rf = rf_model.predict(features_scaled)[0]

    # Ensemble by majority vote
    preds = [pred_svm, pred_dt, pred_rf]
    final_pred = Counter(preds).most_common(1)[0][0]
    final_label = label_encoder.inverse_transform([final_pred])[0]

    st.success(f"Predicted Infection Type: **{final_label}**")



2025-04-06 15:58:58.888 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-06 15:58:59.243 
  command:

    streamlit run C:\Users\krish\anaconda3\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-04-06 15:58:59.244 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-06 15:58:59.245 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-06 15:58:59.246 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-06 15:58:59.247 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-06 15:58:59.249 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-06 15:58:59.250 Thread 'MainThread': mi

In [ ]:
!pip install numpy pandas matplotlib scikit-learn opencv-python scikit-image joblib streamlit
